In [ ]:
import xarray as xr
import datacube_tools
import numpy as np
import matplotlib.pyplot as plt
import re
import os
dc = datacube_tools.DATACUBETOOLS()


## Import datacubes from ITS_LIVE V01 to get some useful bounds

In [ ]:
# Import directory in which this notebook is stored
import os
path = os.getcwd()


# Get the path of the folder SVD_Code
path = re.sub('SVD_Code', '', path)

# Set Data path
data_path = path + 'Data/'

## Import template datasets

In [ ]:
# Import current path 
import os
cwd = os.getcwd()

chunks = {'mid_date': 1000, 'x':100, 'y':100}  # chunk for lazy memory loading
ds_32607 = xr.open_dataset(f'{data_path}/MalaspinaGlacierCube_32607.nc',chunks=chunks) # Open the main dataset
ds_32608 = xr.open_dataset(f'{data_path}/MalaspinaGlacierCube_32608.nc',chunks=chunks) # Open the secondary dataset (smaller in size)

/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "mid_date" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimens

## Create a bounding box from these to download the V02 ITS_LIVE Data

In [4]:
# Get max date of both datasets
sdate = np.max([np.max(ds_32608.mid_date.values), np.max(ds_32607.mid_date.values)])
sdate = np.min([np.min(ds_32608.mid_date.values), np.min(ds_32607.mid_date.values)])
bbox = []

# Get bounding box
bbox.append(np.min(ds_32607.x.values))
bbox.append(np.max(ds_32607.x.values))
bbox.append(np.min(ds_32607.y.values))
bbox.append(np.max(ds_32607.y.values))

import pyproj

# Define the EPSG codes for the source and target coordinate reference systems
src_crs = 'EPSG:32607'
dst_crs = 'EPSG:3413'

# Define the transformer from the source CRS to the target CRS
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True)

# Perform the coordinate transformation
xmin_3413, ymin_3413 = transformer.transform(bbox[0], bbox[2])
xmax_3413, ymax_3413 = transformer.transform(bbox[1], bbox[3])

In [ ]:
def Download_Datacube(var, threshold, bbox, sdate, edate, chunks = 'auto'):

    # Get the ITS_LIVE datacube url for the bounding box
    urls = []
    for i in [0,1]:
        for j in [2,3]:
            cubefeature, bbox_centrer_point_cubexy = dc.find_datacube_catalog_entry_for_point((bbox[i],bbox[j]), '32607')
            urls.append(cubefeature['properties']['zarr_url'])
    urls = list(set(urls)) # Remove duplicates

    cubes = [] # List to store the datacubes
    size = 0
    i = 0

    # Select the variables to keep
    variables_to_keep = [var, 'mid_date', 'x', 'y']

    qual = []
    dates = []
    for url in urls:
        # Load indices of slices above the quality threshold
        valid = xr.open_dataset(url, engine='zarr').roi_valid_percentage.values

        # Grab the time values
        t = xr.open_dataset(url, engine='zarr').mid_date.values
        
        # Create a time mask, based on the validity of layers and the custom date-range
        t_mask = np.logical_and(valid>threshold, t>sdate)

        qual.append(t_mask)
        dates.append(t)


        # Open dataset
        ds = xr.open_dataset(url, decode_timedelta=False, engine="zarr", consolidated=True, chunks=chunks)[f"{var}"]


        
        # Drop variables not in the list
        ds_subset = ds.drop_vars([var for var in ds.variables if var not in variables_to_keep])
        
        # Apply the mask to the dataset
        ds = ds.sel(mid_date=t_mask,
                    x=slice(xmin_3413, xmax_3413),
                    y=slice(ymin_3413, ymax_3413))
        
        ds = ds.sortby('mid_date')
        ds = ds.drop_duplicates(dim='mid_date')
    
        
        # Resample and calculate the mean every 5 days
        cubes.append(ds)


    # Assuming your datasets are stored in a list called 'datasets'
    common_dates = cubes[0]['mid_date']  # Start with the dates from the first dataset
    for ds in cubes[1:]:
        common_dates = np.intersect1d(common_dates, ds['mid_date'])  # Find common dates

    # Now filter each dataset to keep only the common dates
    cubes = [ds.sel(mid_date=common_dates) for ds in cubes]

    # Combine the cubes into a single dataset
    cubes = xr.combine_by_coords(cubes)

    # Rechunk the dataset
    cubes = cubes.chunk({'mid_date':1000, 'y': 100, 'x':100})

    # Write the dataset on disk
    from dask.diagnostics import ProgressBar
    write_job = cubes.to_netcdf(f'{data_path}single_threader{var}.nc', compute=False)
    with ProgressBar():
        print(f"Writing to {f'{data_path}single_threader{var}.nc'}")
        write_job.compute(scheduler='single-threaded')

/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/indexing.py:1617: PerformanceWarning: Slicing with an out-of-order index is generating 5067 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/indexing.py:1617: PerformanceWarning: Slicing with an out-of-order index is generating 2353 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/indexing.py:1617: PerformanceWarning: Slicing with an out-of-order index is generating 10372 times more chunks
  return self.array[key]


In [ ]:
# Set data threshold
threshold = 40 # Slices with less than 40% of valid data will be discarded

# Set the variable to download
var = 'vx' # Choose between 'v', 'vx', 'vy'
Download_Datacube(var, threshold, bbox, sdate, edate, chunks)

Writing to test_single_threadervy.nc
[                                        ] | 0% Completed | 242.04 us

IOStream.flush timed out


[                                        ] | 0% Completed | 232.75 s